In [2]:

import os
import matplotlib
from matplotlib import pyplot
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
# be able to save images on server
matplotlib.use('Agg')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if __name__ == '__main__':


In [3]:
pip install sqldf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Air quality anno 2014-2020

In [ ]:
airQuality = read_csv("AirQualitySchivenoglia14-20.csv")
airQuality.drop(["CO", "NO2", "NOx", "Ozone",
                "Sulfur_dioxide"], axis=1, inplace=True)


airQuality


,IDStation,NameStation,Date,Ammonia,Arsenic,Benzene,Benzo_a_pyrene,Cadmium,Lead,Nikel,PM10,PM25
0,703,Schivenoglia Via Malpasso,2014-01-01,14.1,NaN,0.6,1.5,NaN,NaN,NaN,56.0,56.0
1,703,Schivenoglia Via Malpasso,2014-01-02,19.4,1.1,0.8,NaN,0.0,4.2,4.2,56.0,55.0
2,703,Schivenoglia Via Malpasso,2014-01-03,10.2,NaN,0.5,NaN,NaN,NaN,NaN,43.0,39.0
3,703,Schivenoglia Via Malpasso,2014-01-04,8.6,NaN,0.7,0.8,NaN,NaN,NaN,37.0,33.0
4,703,Schivenoglia Via Malpasso,2014-01-05,7.7,NaN,0.8,NaN,NaN,NaN,NaN,18.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2547,703,Schivenoglia Via Malpasso,2020-12-27,NaN,NaN,0.8,NaN,NaN,NaN,NaN,23.0,21.0
2548,703,Schivenoglia Via Malpasso,2020-12-28,NaN,NaN,0.8,NaN,NaN,NaN,NaN,18.0,14.0
2549,703,Schivenoglia Via Malpasso,2020-12-29,5.0,NaN,1.2,NaN,NaN,NaN,NaN,33.0,32.0
2550,703,Schivenoglia Via Malpasso,2020-12-30,5.9,NaN,1.7,NaN,NaN,NaN,NaN,52.0,48.0


## Weather 2014-2015

In [ ]:
w14_15 = read_csv("WeatherSchivenoglia14-15.csv")
w16_17 = read_csv("WeatherSchivenoglia16-17.csv")
w18_19 = read_csv("WeatherSchivenoglia18-19.csv")
w20 = read_csv("WeatherSchivenoglia20.csv")


### Unione delle tabelle del meteo

In [ ]:
weather = sqldf('''
                SELECT *
                FROM w14_15
                UNION
                 SELECT *
                FROM w16_17
                UNION
                SELECT *
                FROM w18_19
                UNION
                SELECT *
                FROM w20  
                ''')
a = 730+726+729+366
print("Check per vedere se le dimensioni tornano: "+str(a))
weather


Check per vedere se le dimensioni tornano: 2551


,IDStation,NameStation,Date,Wind_speed,Wind_direction,Temperature,Relative_humidity,Global_radiation,Rainfall
0,671,Mantova Tridolino,2014-01-01,1.508213,260.0,4.766667,84.158333,79.737500,0.2
1,671,Mantova Tridolino,2014-01-02,0.390941,332.0,5.479167,87.091667,13.875000,2.0
2,671,Mantova Tridolino,2014-01-03,2.104680,267.0,5.766667,97.412500,23.129167,1.4
3,671,Mantova Tridolino,2014-01-04,0.949083,104.0,6.950000,99.608333,4.858333,5.8
4,671,Mantova Tridolino,2014-01-05,1.130128,248.0,7.541667,99.829167,5.970833,14.2
...,...,...,...,...,...,...,...,...,...
2524,671,Mantova Tridolino,2020-12-27,0.519307,311.0,1.945833,84.305556,81.204861,0.0
2525,671,Mantova Tridolino,2020-12-28,1.756182,351.0,1.222222,99.542361,8.741667,7.0
2526,671,Mantova Tridolino,2020-12-29,1.019866,50.0,2.134722,99.568056,25.305556,0.4
2527,671,Mantova Tridolino,2020-12-30,2.195260,255.0,2.540278,99.589583,18.256944,0.0


# Join per creare la tabella che ci interessa

In [ ]:
final = sqldf('''SELECT *
         FROM airQuality AS a JOIN weather AS w
         ON a.Date =  w.Date
         ''')

print("Dimensioni qualità aria: "+str(airQuality.shape))
print("Dimensioni meteo: "+str(weather.shape))
print("Dimensioni matrice finale"+str(final.shape))
final


Dimensioni qualità aria: (2552, 12)
Dimensioni meteo: (2529, 9)
Dimensioni matrice finale(2524, 21)


,IDStation,NameStation,Date,Ammonia,Arsenic,Benzene,Benzo_a_pyrene,Cadmium,Lead,Nikel,...,PM25,IDStation,NameStation,Date,Wind_speed,Wind_direction,Temperature,Relative_humidity,Global_radiation,Rainfall
0,703,Schivenoglia Via Malpasso,2014-01-01,14.1,NaN,0.6,1.5,NaN,NaN,NaN,...,56.0,671,Mantova Tridolino,2014-01-01,1.508213,260.0,4.766667,84.158333,79.737500,0.2
1,703,Schivenoglia Via Malpasso,2014-01-02,19.4,1.1,0.8,NaN,0.0,4.2,4.2,...,55.0,671,Mantova Tridolino,2014-01-02,0.390941,332.0,5.479167,87.091667,13.875000,2.0
2,703,Schivenoglia Via Malpasso,2014-01-03,10.2,NaN,0.5,NaN,NaN,NaN,NaN,...,39.0,671,Mantova Tridolino,2014-01-03,2.104680,267.0,5.766667,97.412500,23.129167,1.4
3,703,Schivenoglia Via Malpasso,2014-01-04,8.6,NaN,0.7,0.8,NaN,NaN,NaN,...,33.0,671,Mantova Tridolino,2014-01-04,0.949083,104.0,6.950000,99.608333,4.858333,5.8
4,703,Schivenoglia Via Malpasso,2014-01-05,7.7,NaN,0.8,NaN,NaN,NaN,NaN,...,16.0,671,Mantova Tridolino,2014-01-05,1.130128,248.0,7.541667,99.829167,5.970833,14.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2519,703,Schivenoglia Via Malpasso,2020-12-27,NaN,NaN,0.8,NaN,NaN,NaN,NaN,...,21.0,671,Mantova Tridolino,2020-12-27,0.519307,311.0,1.945833,84.305556,81.204861,0.0
2520,703,Schivenoglia Via Malpasso,2020-12-28,NaN,NaN,0.8,NaN,NaN,NaN,NaN,...,14.0,671,Mantova Tridolino,2020-12-28,1.756182,351.0,1.222222,99.542361,8.741667,7.0
2521,703,Schivenoglia Via Malpasso,2020-12-29,5.0,NaN,1.2,NaN,NaN,NaN,NaN,...,32.0,671,Mantova Tridolino,2020-12-29,1.019866,50.0,2.134722,99.568056,25.305556,0.4
2522,703,Schivenoglia Via Malpasso,2020-12-30,5.9,NaN,1.7,NaN,NaN,NaN,NaN,...,48.0,671,Mantova Tridolino,2020-12-30,2.195260,255.0,2.540278,99.589583,18.256944,0.0


# Export del file 

In [ ]:
os.chdir('/Users/marcovinciguerra/Github/GitTesi/Python/Schivenoglia')


In [ ]:
final.to_csv('RisultatiSchivenoglia.csv')


In [ ]:
os.chdir('/Users/marcovinciguerra/Github/GitTesi/Python/Schivenoglia/QuadrantiSchivenoglia')
final.to_csv('RisultatiSchivenoglia.csv')
